In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
%matplotlib inline

In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

In [3]:
MAX_TRAINING = 10000
MAX_VALIDATION = 2000
MAX_TESTING = 2000
correct_class = {1 : 'airplane', 2 : 'automobile', 3 : 'bird', 4 : 'cat', 5 : \
                 'deer', 6 : 'dog', 7 : 'frog', 8 : 'horse', 9 : 'ship', 10 : 'truck'}

In [4]:
#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()

# Keeping the data between 0 and 1
x_train_max, x_test_max = x_train / 255.0, x_test / 255.0
y_train_max = to_categorical(y_train, num_classes=10)
y_test_max = to_categorical(y_test, num_classes=10)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, MAX_TRAINING*3, MAX_TRAINING)
x_val, y_val = get_random_data(x_train_max, y_train_max, MAX_TRAINING*3+1, x_train_max.shape[0], MAX_VALIDATION)
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=10000, Y=10000
Number of Validation Samples: X=2000, Y=2000
Number of Test Samples: X=2000, Y=2000


In [5]:
class MetricsAfterEachEpoch(tf.keras.callbacks.Callback):
    def on_train_begin(self, scores={}):
        self.loss, self.accuracy, self.mae, self.mse = [], [], [], []
    def on_train_end(self, logs={}):
        return self.loss, self.accuracy, self.mae, self.loss
    def on_epoch_end(self, epoch, scores):
        self.loss.append(scores.get('loss'))
        self.accuracy.append(scores.get('acc'))
        self.mae.append(scores.get('mean_absolute_error'))
        self.loss.append(scores.get('mean_squared_error'))
        return

In [6]:
def mini_vgg_net(activation_func):
    # first CONV => RELU => CONV => RELU => POOL layer set
    # second CONV => RELU => CONV => RELU => POOL layer set
    # first (and only) set of FC => RELU layers
    model = Sequential()

    #Instantiating first set of Layers
    model.add(Conv2D(32, kernel_size=(3, 3), activation=activation_func, padding='same'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation=activation_func, padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1), name='pool1'))

    # Instantiating second set of Layers
    model.add(Conv2D(64, kernel_size=(3, 3), activation=activation_func, padding='same'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation=activation_func, padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1), name='pool2'))


    # Instantiating set of FCs
    model.add(Flatten())
    model.add(Dense(512, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    
    return model

In [7]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=25):
    model = mini_vgg_net(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=128)
    
    #Saving the model
    if label == 'RAdam':
        model.save("../saved_models/{}_{}_{}.hdf5".format('RAdam', activation_func, "MiniVGGNet"))
    else: 
        model.save("../saved_models/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, "MiniVGGNet"))
    return score, history

In [ ]:
testing_optimizers = ['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh', 'relu']
loss, accuracy, mae, mse = {}, {}, {}, {}
train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

for optimizer in testing_optimizers:
    for activation in testing_activations:
        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.0005), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)
        
        #Test Side Stats
        loss[(optimizer, activation)] = score_returned[0]
        accuracy[(optimizer, activation)] = score_returned[1]
        mae[(optimizer, activation)] = score_returned[2]
        mse[(optimizer, activation)] = score_returned[3]
        
        #Training Side Stats
        train_loss_dict[(optimizer, activation)] = history.get('loss')
        train_accuracy_dict[(optimizer, activation)] = history.get('acc')
        train_mae_dict[(optimizer, activation)] = history.get('mean_absolute_error')
        train_mse_dict[(optimizer, activation)] = history.get('mean_squared_error')
        
        bg = (loss, accuracy, mae, mse, train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict)
        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, 'MiniVGGNet', 'milestone-experiments', 'benchmark_testing')
        
        writeToDB(optimizer, activation, 'MiniVGGNet', bg, 'milestone-experiments', 'benchmark_testing')

Epoch 1/25


In [ ]:
print (loss, accuracy)

In [ ]:
accuracy

In [ ]:
mae

In [ ]:
mse

In [ ]:
train_loss_dict

In [ ]:
train_accuracy_dict

In [ ]:
train_mae_dict

In [ ]:
train_mse_dict